# ParamRef ref-first parameter system — Comprehensive test & demo notebook

This notebook is a **test harness** and **interactive demonstration** for the ref-first parameter system. It is intended to do two things:

1. **Illustrate usage** (interactive controls, hooks, plots).
2. **Validate behavior** (assertions and small “unit/integration” style tests in-notebook).

## What is tested

The notebook is organized around the design requirements:

- **Ref-first API**: `fig.params[sym] -> ParamRef` (not widgets).
- **Single entrypoint**: `fig.parameter(...)` is the *only* creation/config policy; no `ensure(...)`.
- **Idempotent updates**: calling `parameter(...)` again updates range/value settings.
- **Normalized events**: `ParamRef.observe(cb)` yields `ParamEvent`.
- **Render routing (Option A)**: param change → `SmartFigure.render("param_change", event)` → plots → hooks.
- **Reset semantics**: reset is **value-only**, and `parameter(..., value=v)` updates the reset default value.
- **Multi-symbol handshake**: `parameter([a,b], control=pad)` calls `pad.make_refs([a,b])` once (if a pad exists).
- **UI duplication prevention**: repeated parameter calls do not add duplicate controls to the UI container.

---

## Running notes

- Many checks are written with plain `assert` statements so failures are immediate and local.
- Some sections are **optional** (e.g. multi-symbol control). They are skipped if you do not configure the relevant adapter entries.

## 0. Environment and dependencies

This notebook expects (at minimum):

- `sympy`
- `ipywidgets` (for interactive controls)
- your project package installed in the environment (editable install recommended)

If `ipywidgets` does not render in your environment, you can still run the *non-UI* tests; the interactive demo cells will be less meaningful.

In [1]:
import sys
import importlib
import inspect
from dataclasses import is_dataclass
from typing import Any, Callable, Dict, Optional, Sequence, Tuple

import sympy as sp

try:
    import ipywidgets as widgets
    from IPython.display import display
    _HAS_WIDGETS = True
except Exception as e:
    _HAS_WIDGETS = False
    widgets = None
    display = None
    print("WARNING: ipywidgets could not be imported; interactive UI cells may not work.")
    print("Import error:", repr(e))

print("Python:", sys.version)
print("sympy:", sp.__version__)
print("ipywidgets available:", _HAS_WIDGETS)

Python: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]
sympy: 1.14.0
ipywidgets available: True


---

## 1. Setup / Adapter layer (required)

You must configure how this notebook imports and constructs your objects.

### 1.1 Minimal required objects

You must provide:

- `SmartFigure` class (or a factory to create a figure)
- `ParamEvent` type (dataclass recommended)
- Default slider control class (e.g. `SmartFloatSlider`) **or** some control used when `control=None`
- A `SmartFigure()` function returning a fresh figure instance

### 1.2 Optional objects

If you want the multi-symbol control tests:

- a multi-symbol `Control` instance/class (e.g. `SmartPad2D`)
- it must implement `make_refs(symbols)` once-per-call behavior

### 1.3 Import helper

Below is a helper to import objects via a string of the form:

- `"package.module:Name"`

You can either:
- edit the `IMPORTS` dict below, or
- directly write normal Python imports and assign variables.

In [2]:
from pathlib import Path
import sys

# If PACKAGE is not installed, you can add a repo root to sys.path.
# Default assumption: this notebook lives under <repo>/notebooks/... so parents[1] is <repo>.
ROOT = Path.cwd().resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from gu_toolkit.SmartFigure import  *
from gu_toolkit.ParamEvent import  *
from gu_toolkit.SmartSlider import  *

### 1.4 Adapter verification (required)

This cell checks that:

- `SmartFigure()` works,
- `fig.parameter(...)` exists,
- `fig.params[...]` behaves like a mapping,
- `ParamEvent` is importable (strongly recommended).

If this fails, fix the adapter cell above before proceeding.

In [3]:
# --- Adapter sanity checks ---

fig = SmartFigure()
if _HAS_WIDGETS:
    display(fig)
else:
    print("NOTE: ipywidgets not available; skipping figure display.")

assert hasattr(fig, "parameter") and callable(fig.parameter), "Figure must have .parameter(...)"
assert hasattr(fig, "params"), "Figure must expose .params"

params = fig.params

# Mapping-like checks
assert hasattr(params, "__getitem__"), "fig.params must support __getitem__"
assert hasattr(params, "keys") and hasattr(params, "values"), "fig.params should look like a Mapping"

if ParamEvent is None:
    print("WARNING: ParamEvent could not be imported. Event normalization checks will be limited.")
else:
    # Recommended: ParamEvent should be a dataclass (not required).
    if not is_dataclass(ParamEvent):
        print("WARNING: ParamEvent is not a dataclass (allowed, but some checks may be weaker).")

print("Adapter sanity checks passed.")


OneShotOutput()

Adapter sanity checks passed.


---

## 2. Shared utilities for testing

The notebook uses a few helpers:

- `expect_raises(...)` for exception assertions
- `CounterCallback` to record events
- `try_call(...)` to call APIs with mild signature variation (useful if your hook API differs slightly)

In [4]:
import contextlib

@contextlib.contextmanager
def expect_raises(exc_type):
    try:
        yield
    except exc_type:
        return
    except Exception as e:
        raise AssertionError(f"Expected {exc_type.__name__}, got {type(e).__name__}: {e}") from e
    raise AssertionError(f"Expected {exc_type.__name__} to be raised, but no exception occurred.")

class CounterCallback:
    def __init__(self):
        self.calls = 0
        self.events = []

    def __call__(self, event):
        self.calls += 1
        self.events.append(event)

def try_call(func, *args, **kwargs):
    """Try calling func with kwargs; if TypeError, progressively drop unknown kwargs."""
    try:
        return func(*args, **kwargs)
    except TypeError:
        # Drop kwargs one by one in a stable order
        for k in list(kwargs.keys()):
            kw2 = dict(kwargs)
            kw2.pop(k)
            try:
                return func(*args, **kw2)
            except TypeError:
                pass
        raise

def observe_call(ref, cb, *, fire=False):
    """Call ref.observe(cb, fire=...) if supported; otherwise fall back to ref.observe(cb)."""
    try:
        return ref.observe(cb, fire=fire)
    except TypeError:
        # Some implementations do not support the 'fire' kwarg.
        return ref.observe(cb)


---

## 3. Baseline: create a parameter and verify ref-first API

### Required behavior
- `fig.parameter(a)` returns a `ParamRef`
- `fig.params[a]` returns a `ParamRef`
- the ref exposes `.value` and `.observe(...)`

In [5]:
fig = SmartFigure()
a = sp.Symbol("a")

ref_a = fig.parameter(a)
assert ref_a is fig.params[a], "fig.parameter(a) should return the same ref stored in fig.params[a]"
assert hasattr(ref_a, "value"), "ParamRef must expose .value"
assert hasattr(ref_a, "observe") and callable(ref_a.observe), "ParamRef.observe must exist and be callable"

print("Ref-first API baseline passed.")

Ref-first API baseline passed.


### Optional: show the UI widget for `a`

If your implementation supports `fig.params.widget(a)` or `ref.widget`, this cell displays it.

In [6]:
fig = SmartFigure()
a = sp.Symbol("a")
ref_a = fig.parameter(a)

widget_obj = None
if hasattr(fig.params, "widget") and callable(fig.params.widget):
    try:
        widget_obj = fig.params.widget(a)
    except Exception as e:
        print("fig.params.widget(a) exists but failed:", repr(e))

if widget_obj is None and hasattr(ref_a, "widget"):
    widget_obj = getattr(ref_a, "widget", None)

if _HAS_WIDGETS and widget_obj is not None:
    display(widget_obj)
else:
    print("Widget display skipped (no widget available or ipywidgets not available).")

SmartFloatSlider(children=(HBox(children=(HTML(value='\n<style>\n/* ==========================================…

---

## 4. Idempotence: configuration updates on repeated `parameter(...)`

### Required behavior
Calling `parameter(a, ...)` again must **apply provided kwargs** even if `a` already exists.

This section checks typical slider capabilities (`min`, `max`, `step`, `value`) if they exist.

If your ref does not support a capability, it should raise `AttributeError` for that attribute.

In [7]:
fig = SmartFigure()
a = sp.Symbol("a")
param_ref = fig.parameter(a)

# Try setting a value; should always be supported.
param_ref.value = 0.25
assert abs(param_ref.value - 0.25) < 1e-12, "ref.value roundtrip failed"

# Now call parameter again with updates (idempotent config)
# We attempt slider-like kwargs; if your default control is not range-capable, this will raise AttributeError.
kwargs = {"min": -2.0, "max": 2.0, "step": 0.1, "value": 0.3}

try:
    ref2 = fig.parameter(a, **kwargs)
    assert ref2 is param_ref, "parameter(a, ...) should not create a new ref"
    assert abs(param_ref.value - 0.3) < 1e-12, "parameter(a, value=...) did not update value"
    # Capability checks (only if available)
    for k in ("min", "max", "step"):
        if hasattr(param_ref, k):
            assert abs(getattr(param_ref, k) - kwargs[k]) < 1e-12, f"{k} was not updated idempotently"
    print("Idempotent update checks passed (range-capable).")
except AttributeError as e:
    print("Range capability not supported by this ParamRef/control (AttributeError):", e)
    # Value update should still work
    ref2 = fig.parameter(a, value=0.3)
    assert abs(param_ref.value - 0.3) < 1e-12, "parameter(a, value=...) must still work even without range capabilities"
    print("Idempotent update checks passed (value-only control).")

Idempotent update checks passed (range-capable).


---

## 5. `ParamRef.observe` and normalized `ParamEvent`

### Required behavior
- `ref.observe(cb)` registers `cb(event)` to fire on effective value changes.
- The event should be a normalized `ParamEvent` (recommended: dataclass) with at least:
  - `parameter`, `old`, `new`, `ref`

This section:
1. attaches a callback to `ref_a`,
2. changes the value,
3. validates callback call count and event structure.

In [8]:
fig = SmartFigure()
a = sp.Symbol("a")
param_ref = fig.parameter(a)

cb = CounterCallback()
observe_call(param_ref, cb, fire=False)

old_val = param_ref.value
new_val = (old_val + 1.0) if isinstance(old_val, (int, float)) else 1.0

param_ref.value = new_val

assert cb.calls >= 1, "observe callback did not fire on value change"
evt = cb.events[-1]

# Structural checks
for attr in ("parameter", "old", "new", "ref"):
    assert hasattr(evt, attr), f"ParamEvent must have attribute {attr!r}"

assert evt.parameter == a, "event.parameter must match symbol"
assert evt.ref is param_ref, "event.ref must reference the same ParamRef instance"

print("observe(...) fired and produced an event with required fields.")


observe(...) fired and produced an event with required fields.


### 5.1 `fire=True` semantics (recommended)

`ref.observe(cb, fire=True)` should immediately call `cb` once with a synthetic event reflecting current value.

In [9]:
fig = SmartFigure()
a = sp.Symbol("a")
param_ref = fig.parameter(a)

cb = CounterCallback()
observe_call(param_ref, cb, fire=True)

# If fire=True is not supported, observe_call falls back to observe(cb) and this test is skipped.
if cb.calls == 0:
    print("Skipping fire=True semantics test (this ParamRef.observe does not support fire=True).")
else:
    assert cb.calls >= 1, "fire=True did not trigger immediate callback"
    evt = cb.events[0]
    assert evt.parameter == a
    assert evt.new == param_ref.value, "fire=True event should reflect current value as 'new'"
    print("fire=True semantics appear to be implemented.")


fire=True semantics appear to be implemented.


---

## 6. Render routing (Option A): param change triggers render and hooks

### Design requirement
Parameter changes should route centrally:

`ParamRef.observe → ParameterManager._on_param_change → SmartFigure.render("param_change", event)`.

This notebook cannot assume your internal wiring, so we validate routing using *observable outcomes*:

- Hooks run on parameter changes (and run after plots are rendered).
- Optionally, we also “spy” on `fig.render` if it is callable and can be wrapped.

### 6.1 Hook presence and adapter

This cell builds small adapters for hook registration because some projects differ slightly in signature.

Expected (from spec):
- `fig.add_hook(callback, run_now=True)`.

If your API differs, update `register_hook` below.

In [10]:
def register_hook(fig, cb, run_now=True):
    # Preferred name in this codebase
    if hasattr(fig, "add_hook") and callable(fig.add_hook):
        return try_call(fig.add_hook, cb, run_now=run_now)
    # Alternate name (supported by SmartFigure.py in some versions)
    if hasattr(fig, "add_param_change_hook") and callable(fig.add_param_change_hook):
        return try_call(fig.add_param_change_hook, cb, run_now=run_now)
    raise RuntimeError("Figure does not expose add_hook(...) or add_param_change_hook(...).")

# Quick sanity check
fig = SmartFigure()
assert hasattr(fig, "render") and callable(fig.render), "Figure must have a render(...) method"

print("Hook adapter ready.")


Hook adapter ready.


### 6.2 Hook fires after a parameter change

We register a hook, change a parameter value, and check that the hook was called.

In [11]:
fig = SmartFigure()
a = sp.Symbol("a")
param_ref = fig.parameter(a)

hook = CounterCallback()
register_hook(fig, hook, run_now=False)

param_ref.value = (param_ref.value + 0.5) if isinstance(param_ref.value, (int, float)) else 0.5

assert hook.calls >= 1, "Hook did not fire after parameter change (render routing may be broken)."
print("Hook fired after parameter change.")

Hook fired after parameter change.


---

## 7. Reset semantics (value-only) and “reset default” update via `parameter(..., value=...)`

### Required behavior
- `ref.reset()` resets **value only**, not `min/max/step`.
- Calling `parameter(a, value=v)` updates what reset returns to (for slider-like controls).

This section runs two tests:
1. reset does not change range config (if range exists)
2. reset returns to the last `parameter(..., value=...)` configured value

In [38]:
param_ref =fig.parameter(a)
param_ref.value
param_ref.value = 1.234
param_ref.value
param_ref.reset()
param_ref.value

0.0

In [13]:
fig = SmartFigure()
a = sp.Symbol("a")
param_ref = fig.parameter(a)

# Set a baseline configured value via parameter(...)
fig.parameter(a, value=0.7)
assert abs(param_ref.value - 0.7) < 1e-12

# If min/max exist, change them and ensure reset does not revert them.
min_before = getattr(param_ref, "min", None) if hasattr(param_ref, "min") else None
max_before = getattr(param_ref, "max", None) if hasattr(param_ref, "max") else None

if hasattr(param_ref, "min") and hasattr(param_ref, "max"):
    fig.parameter(a, min=-3.0, max=3.0)
    min_after = param_ref.min
    max_after = param_ref.max
else:
    min_after = max_after = None

# Move value away, then reset
param_ref.value = 1.234

assert abs(param_ref.value - 1.234) < 1e-12
assert hasattr(param_ref, "reset") and callable(param_ref.reset), "ParamRef.reset() is required"
param_ref.reset()


assert abs(param_ref.value - 0.7) < 1e-12, "reset() did not return value to configured default value"

# Range unchanged by reset (only if range is supported)
if hasattr(param_ref, "min") and hasattr(param_ref, "max"):
    assert param_ref.min == min_after and param_ref.max == max_after, "reset() must not revert min/max/step"

print("Reset semantics checks passed.")

AssertionError: reset() did not return value to configured default value

---

## 9. UI duplication prevention (behavioral)

### Requirement
Repeated calls to `parameter(a)` must not add duplicate controls to the UI container.

Because the internal UI container differs across projects, this section checks **public/UI-facing surfaces** first:

- `fig.params.widgets()` if present should not grow on repeated idempotent calls.

If you do not have `widgets()`, this test is limited.

In [39]:
fig = SmartFigure()
a = sp.Symbol("a")

fig.parameter(a)
fig.parameter(a)  # repeat

if hasattr(fig.params, "widgets") and callable(fig.params.widgets):
    ws1 = fig.params.widgets()
    fig.parameter(a)  # repeat again
    ws2 = fig.params.widgets()
    assert len(ws2) == len(ws1), "widgets() grew after repeated parameter(a) calls"
    print("widgets() did not grow on repeated calls.")
else:
    print("Skipping widgets() duplication test: fig.params.widgets() not available.")

widgets() did not grow on repeated calls.


---

## 10. “No backdoor creation” via plots (optional integration)

This section is intentionally conservative because plot APIs vary.

You should adapt `make_plot_that_requires_symbol(fig, a)` so that:

- it registers a plot that uses symbol `a` (e.g. expression evaluation),
- and triggers parameter inference/creation if your system supports it.

The requirement is:

- plot inference must call `fig.parameter(...)`, not `ensure(...)`.

We validate it behaviorally by checking that `a` ends up in `fig.params` after plot creation.

In [40]:
def make_plot_that_requires_symbol(fig, sym):
    """Create a plot that uses `sym` with parameters=None so inference should create it."""
    x = sp.Symbol("x")
    # Ensure sym != x
    if sym == x:
        raise ValueError("sym must be distinct from the plotting variable.")
    fig.plot(x, sym * sp.sin(x), parameters=None, id="__inference_test__")
    return True

fig = SmartFigure()
a = sp.Symbol("a")

installed = make_plot_that_requires_symbol(fig, a)
if not installed:
    raise AssertionError("make_plot_that_requires_symbol must return True in this adapted notebook.")
else:
    # After installing plot, the symbol should be registered via fig.parameter(...)
    try:
        _ = fig.params[a]
        print("Plot inference created the parameter via the public surface.")
    except KeyError:
        raise AssertionError("Plot inference did not create parameter; or creation path is not wired.")


Plot inference created the parameter via the public surface.


---

## 11. Manual interactive demo (optional)

If you have `ipywidgets` and your controls are widget-backed, this section creates a small interactive demonstration:

- display widgets for parameters `a` (and optionally `b`)
- register a hook and print events as you interact

This is not a strict automated test; it is intended for **human verification**.

In [41]:
fig = SmartFigure()
a = sp.Symbol("a")
b = sp.Symbol("b")

ref_a = fig.parameter(a, value=0.0)
# Create b only if you want
# ref_b = fig.parameter(b, value=0.0)

def pretty_print_event(evt):
    # Robust printing without assuming exact types
    attrs = {}
    for k in ("parameter", "old", "new"):
        if hasattr(evt, k):
            attrs[k] = getattr(evt, k)
    print("Event:", attrs)

# Register param observers
cb = CounterCallback()
ref_a.observe(cb, fire=True)

# Register a hook that prints
def hook_printer(evt):
    print("[hook]")
    pretty_print_event(evt)

try:
    register_hook(fig, hook_printer, run_now=True)
except Exception as e:
    print("Hook registration failed:", repr(e))

# Display UI if possible
widget_obj = None
if hasattr(fig.params, "widget") and callable(fig.params.widget):
    try:
        widget_obj = fig.params.widget(a)
    except Exception:
        widget_obj = None
if widget_obj is None and hasattr(ref_a, "widget"):
    widget_obj = getattr(ref_a, "widget", None)

if _HAS_WIDGETS and widget_obj is not None:
    display(widget_obj)
    print("Interact with the widget; events should print above.")
else:
    print("Interactive demo skipped (no widget or ipywidgets unavailable).")

[hook]
Event: {}


SmartFloatSlider(children=(HBox(children=(HTML(value='\n<style>\n/* ==========================================…

Interact with the widget; events should print above.


---

## 12. Summary of what to do when a test fails

This notebook is designed so that failures indicate a small set of likely causes.

### 12.1 Ref-first failures
- `fig.parameter(a)` returns a widget: you are returning a control rather than a `ParamRef`.
- `fig.params[a]` returns something else: manager storage is not ref-first.

### 12.2 Idempotence failures
- `parameter(a, min=..., value=...)` doesn’t update existing: your `parameter(...)` method is returning early (old `ensure` behavior).
- `value` applied before range: reorder application to apply `min/max/step` before `value`.

### 12.3 Observe / event failures
- callback never fires: ref isn’t wiring widget events, or value setting bypasses the observed property.
- event fields missing: ensure normalization to a `ParamEvent`-like object.

### 12.4 Render routing failures
- hook not firing on parameter changes: manager internal observe wiring is missing, or `_on_param_change` is not calling render callback.

### 12.5 Reset failures
- reset restores min/max/step: reset must be value-only; adjust implementation or wrapper behavior.
- reset returns to an old default: ensure `parameter(..., value=v)` updates the stored reset default value.

---

## 13. Next step: export the tests to CI

Many “assert-only” cells can be converted into a `pytest` test module.

A practical pattern:
- keep the adapter in a `conftest.py` fixture that constructs figures,
- reuse the same test logic for CI.